In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(plyr)
library(dplyr)
library(gridExtra)
library(GGally)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [2]:
student_p <- tempfile () #Creating temporary file name inorder to download the zip folder
download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip", student_p)
unzip(student_p, "student-por.csv") #unzip the specific file in zip folder
unlink(student_p) #delete the temporary file 
por_data <- read.csv("student-por.csv", sep = ";") #read file as .csv with seperation of ";"

In [3]:
student_m <- tempfile () 
download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip", student_m)
unzip(student_m, "student-mat.csv")
unlink(student_m)
mat_data <- read.csv("student-mat.csv", sep = ";")

In [4]:
mat_data_1 <- mat_data %>% select(G3, studytime, schoolsup, internet, absences) %>% #select variables will be used in analysis
    mutate(studytime = as.factor(studytime)) %>%
    mutate(schoolsup = as.factor(schoolsup)) %>%
    mutate(internet = as.factor(internet)) %>%
    mutate(absences = as.numeric(absences)) # correct the label for variables
glimpse(mat_data_1) #show number of observations 

Rows: 395
Columns: 5
$ G3        <int> 6, 6, 10, 15, 10, 15, 11, 6, 19, 15, 9, 12, 14, 11, 16, 14,…
$ studytime <fct> 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 3, 1, 3, 2, 1, 1,…
$ schoolsup <fct> yes, no, yes, no, no, no, no, yes, no, no, no, no, no, no, …
$ internet  <fct> no, yes, yes, yes, no, yes, yes, no, yes, yes, yes, yes, ye…
$ absences  <dbl> 6, 4, 10, 2, 4, 10, 0, 6, 0, 0, 0, 4, 2, 2, 0, 4, 6, 4, 16,…


The number of observations for student performance in math dataset is 395 (minus the row of variable name). 

In [5]:
mat_split <- initial_split(mat_data_1, prop = 0.75, strata = G3) #split training dataset, 75%
mat_train <- training(mat_split) #training data
stat <- c("Min.","1st Qu.", "Median", "Mean", "3rd Qu.", "Max.")
sum_mat_G3 <- summary(mat_train$G3)
sum_mat_absences <- summary(mat_train$absences)
tibble(stat, sum_mat_G3, sum_mat_absences) #summary of two numeric variables in math training dataset

stat,sum_mat_G3,sum_mat_absences
<chr>,<table>,<table>
Min.,0.00,0.000
1st Qu.,8.00,0.000
Median,11.00,4.000
Mean,10.49,5.738
3rd Qu.,14.00,8.000
Max.,20.00,75.000


In [6]:
por_data_1 <- por_data %>% select(G3, studytime, schoolsup, internet, absences) %>%
    mutate(studytime = as.factor(studytime)) %>%
    mutate(schoolsup = as.factor(schoolsup)) %>%
    mutate(internet = as.factor(internet)) %>%
    mutate(absences = as.numeric(absences))
glimpse(por_data_1)

Rows: 649
Columns: 5
$ G3        <int> 11, 11, 12, 14, 13, 13, 13, 13, 17, 13, 14, 13, 12, 13, 15,…
$ studytime <fct> 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 3, 1, 3, 2, 1, 1,…
$ schoolsup <fct> yes, no, yes, no, no, no, no, yes, no, no, no, no, no, no, …
$ internet  <fct> no, yes, yes, yes, no, yes, yes, no, yes, yes, yes, yes, ye…
$ absences  <dbl> 4, 2, 6, 0, 0, 6, 0, 2, 0, 0, 2, 0, 0, 0, 0, 6, 10, 2, 2, 6…


The number of observations for student performance in Portuguese dataset is 648 (minus the row of variable name).

In [7]:
set.seed(100)
por_split <- initial_split(por_data_1, prop = 0.6, strata = G3)
por_train <- training(por_split) #training data
sum_por_G3 <- summary(mat_train$G3)
sum_por_absences <- summary(mat_train$absences)
tibble(stat, sum_mat_G3, sum_mat_absences) #summary of two numeric variables in por training dataset

stat,sum_mat_G3,sum_mat_absences
<chr>,<table>,<table>
Min.,0.00,0.000
1st Qu.,8.00,0.000
Median,11.00,4.000
Mean,10.49,5.738
3rd Qu.,14.00,8.000
Max.,20.00,75.000
